# Alt Carbon: Anomaly Detection for Pumps



In [ ]:
BASE_DIR = "/content/drive/MyDrive/DSN_AI/"

## Import Libraries

In [ ]:
import librosa
from pylab import plot, show, figure, imshow, xlabel, ylabel, title
 
import pandas as pd
import numpy as np
 
from tqdm import tqdm
import requests
import os
import re
import glob
import sys
import itertools
import joblib
import json
import math 
import random
import warnings
import tensorflow as tf
 
warnings.filterwarnings(action="ignore")
 
s =42
np.random.seed(s)
tf.random.set_seed(s)
random.seed(s)

In [ ]:
try:
    import kerastuner as kt
except:
    !pip install -q -U keras-tuner
    import kerastuner as kt

     |████████████████████████████████| 97 kB 3.6 MB/s 


## Extracting Audio Features

### Helper functions

In [ ]:

# get machine IDs
def get_section_names(target_dir,
                      dir_name,
                      ext="wav"):
    """
    target_dir : str
        base directory path
    dir_name : str
        sub directory name
    ext : str (default="wav)
        file extension of audio files
    return :
        section_names : list [ str ]
            list of section names extracted from the names of audio files
    """
    # create test files
    file_paths = os.listdir(f"{target_dir}/{dir_name}")
    section_names = sorted(list(set(itertools.chain.from_iterable(
        [re.findall('section_[0-9][0-9]', ext_id) for ext_id in file_paths]))))
    return section_names

# get the list of wave file paths
def file_list_generator(target_dir,
                        section_name,
                        dir_name,
                        mode=None,
                        prefix_normal="normal",
                        prefix_anomaly="anomaly",
                        ext="wav"):
    """
    target_dir : str
        base directory path
    section_name : str
        section name of audio file in <<dir_name>> directory
    dir_name : str
        sub directory name
    prefix_normal : str (default="normal")
        normal directory name
    prefix_anomaly : str (default="anomaly")
        anomaly directory name
    ext : str (default="wav")
        file extension of audio files
    return :
        if the mode is "development":
            files : list [ str ]
                audio file list
            labels : list [ boolean ]
                label info. list
                * normal/anomaly = 0/1
        if the mode is "evaluation":
            files : list [ str ]
                audio file list
    """
    # development
    if mode:
        query = os.path.abspath("{target_dir}/{dir_name}/{section_name}_*_{prefix_normal}_*.{ext}".format(target_dir=target_dir,
                                                                                                     dir_name=dir_name,
                                                                                                     section_name=section_name,
                                                                                                     prefix_normal=prefix_normal,
                                                                                                     ext=ext))
        normal_files = sorted(glob.glob(query))
        normal_labels = np.zeros(len(normal_files))

        query = os.path.abspath("{target_dir}/{dir_name}/{section_name}_*_{prefix_normal}_*.{ext}".format(target_dir=target_dir,
                                                                                                     dir_name=dir_name,
                                                                                                     section_name=section_name,
                                                                                                     prefix_normal=prefix_anomaly,
                                                                                                     ext=ext))
        anomaly_files = sorted(glob.glob(query))
        anomaly_labels = np.ones(len(anomaly_files))

        files = np.concatenate((normal_files, anomaly_files), axis=0)
        labels = np.concatenate((normal_labels, anomaly_labels), axis=0)

    # evaluation
    else:
        query = os.path.abspath("{target_dir}/{dir_name}/{section_name}_*.{ext}".format(target_dir=target_dir,
                                                                                                     dir_name=dir_name,
                                                                                                     section_name=section_name,
                                                                                                     ext=ext))
        files = sorted(glob.glob(query))
        labels = None
    return files, labels

### MFCC Extraction

In [ ]:
SAMPLE_RATE = 22050
DURATION = 10
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

In [ ]:
def extract_mfcc(file_path, n_mfcc=128, n_fft=1024, hop_length=512):
  """ Extract MFCC data from a track for use in genre prediction
  """
  signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)
  # process segments, extracting mfccs and storing data 
  print(f'processing mfcc for {file_path}')
  mfcc = librosa.feature.mfcc(signal,
                              #sr=SAMPLE_RATE,
                              n_fft=n_fft,
                              n_mfcc=n_mfcc,
                              hop_length=hop_length, power=2.0
                              )
  mfcc = mfcc.T # A transpose

  return mfcc

### Melspectogram Extraction

In [ ]:
SAMPLE_RATE = 22050
 
# feature extractor
def file_to_vectors(file_name,
                    n_mels=64,
                    n_frames=5,
                    n_fft=1024,
                    hop_length=512,
                    power=2.0):
    """
    convert file_name to a vector array.
    file_name : str
        target .wav file
    return : numpy.array( numpy.array( float ) )
        vector array
        * dataset.shape = (dataset_size, feature_vector_length)
    """
    # calculate the number of dimensions
    dims = n_mels * n_frames
 
    # generate melspectrogram using librosa
    signal, sr = librosa.load(file_name, sr=None, mono=True)
    mel_spectrogram = librosa.feature.melspectrogram(y=signal,
                                                     sr=sr,
                                                     n_fft=n_fft,
                                                     hop_length=hop_length,
                                                     n_mels=n_mels,
                                                     power=power)
 
    # convert melspectrogram to log mel energies
    log_mel_spectrogram = 20.0 / power * np.log10(np.maximum(mel_spectrogram, sys.float_info.epsilon))
    # calculate total vector size
    n_vectors = len(log_mel_spectrogram[0, :]) - n_frames + 1
    # skip too short clips
    if n_vectors < 1:
        return np.empty((0, dims))
 
    # generate feature vectors by concatenating multiframes
    vectors = np.zeros((n_vectors, dims))
    for t in range(n_frames):
        vectors[:, n_mels * t : n_mels * (t + 1)] = log_mel_spectrogram[:, t : t + n_vectors].T
 
    return vectors

## Modelling

### Helper Functions

In [ ]:
 
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
 
def get_dense_autoencoder(input_dim, lr):
    """
    define the keras model
    the model based on the simple dense auto encoder 
    (128*128*128*128*8*128*128*128*128)
    """
 
    x = Input(shape=(input_dim,))
 
    h = Dense(128)(x)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)
 
    h = Dense(128)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)
 
    h = Dense(128)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)
 
    h = Dense(128)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)
 
    h = Dense(8)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)
    
    h = Dense(128)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)
 
    h = Dense(128)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)
 
    h = Dense(128)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)
 
    h = Dense(128)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)
 
    h = Dense(input_dim)(h)
 
    model = Model(inputs=x, outputs=h)
 
    model.compile(optimizer=keras.optimizers.Adam(lr=lr), 
                  loss='mean_squared_error')
 
    return model
def load_model(file_path):
    return keras.models.load_model(file_path, compile=False)
 
def clear_session():
    K.clear_session()

In [ ]:
import gc
# import additional libraries
import scipy.stats
import joblib
 
# visualizer
class Visualizer(object):
    def __init__(self):
        import matplotlib.pyplot as plt
        self.plt = plt
        self.fig = self.plt.figure(figsize=(7, 5))
        self.plt.subplots_adjust(wspace=0.3, hspace=0.3)
 
    def loss_plot(self, loss, val_loss):
        """
        Plot loss curve.
        loss : list [ float ]
            training loss time series.
        val_loss : list [ float ]
            validation loss time series.
        return   : None
        """
        ax = self.fig.add_subplot(1, 1, 1)
        ax.cla()
        ax.plot(loss)
        ax.plot(val_loss)
        ax.set_title("Model loss")
        ax.set_xlabel("Epoch")
        ax.set_ylabel("Loss")
        ax.legend(["Train", "Validation"], loc="upper right")
 
    def save_figure(self, name):
        """
        Save figure.
        name : str
            save png file path.
        return : None
        """
        self.plt.savefig(name)
 
# get data from the list for file paths
def file_list_to_data(file_list,
                      msg="calc...",
                      n_mels=64,
                      n_frames=5,
                      n_hop_frames=1,
                      n_fft=1024,
                      hop_length=512,
                      power=2.0):
    """
    convert the file_list to a vector array.
    file_to_vector_array() is iterated, and the output vector array is concatenated.
    file_list : list [ str ]
        .wav filename list of dataset
    msg : str ( default = "calc..." )
        description for tqdm.
        this parameter will be input into "desc" param at tqdm.
    return : numpy.array( numpy.array( float ) )
        data for training (this function is not used for test.)
        * dataset.shape = (number of feature vectors, dimensions of feature vectors)
    """
    # calculate the number of dimensions
    dims = n_mels * n_frames
 
    # iterate file_to_vector_array()
    for idx in tqdm(range(len(file_list)), desc=msg):
        vectors = file_to_vectors(file_list[idx],
                                                n_mels=n_mels,
                                                n_frames=n_frames,
                                                n_fft=n_fft,
                                                hop_length=hop_length,
                                                power=power)
        vectors = vectors[: : n_hop_frames, :]
        if idx == 0:
            data = np.zeros((len(file_list) * vectors.shape[0], dims), float)
        data[vectors.shape[0] * idx : vectors.shape[0] * (idx + 1), :] = vectors
 
    return data

### Extract Data

In [ ]:
# "development": mode == True
# "evaluation": mode == False
mode = True
 
# load base_directory list
dirs = [f"{BASE_DIR}Dev Data/dev_data_pump/pump"]
model_dir = "Models/MLP_Autoencoder"
 
# loop of the base directory
for idx, target_dir in enumerate(dirs):
    print("\n===========================")
    print(f"[{idx+1}/{len(dirs)}] {target_dir}")
 
    # set path
    machine_type = os.path.split(target_dir)[1]
 
    section_names_file_path = f"{BASE_DIR}{model_dir}/section_names_{machine_type}.pkl"
    # pickle file for storing anomaly score distribution
    
    
    # get section names from wave file names
    section_names =get_section_names(target_dir, dir_name="train")
    unique_section_names = np.unique(section_names)
    n_sections = unique_section_names.shape[0]
    
    # make condition dictionary
    joblib.dump(unique_section_names, section_names_file_path)
 
    # generate dataset
    print("============== DATASET_GENERATOR ==============")
    # number of wave files in each section
    # required for calculating y_pred for each wave file
    n_files_ea_section = []
        
    for section_idx, section_name in enumerate(unique_section_names):
        # get file list for each section
        # all values of y_true are zero in training
        files, y_true = file_list_generator(target_dir=target_dir,
                                                section_name=section_name,
                                                dir_name="train",
                                                mode=mode)
 
        n_files_ea_section.append(len(files))
 
        data = file_list_to_data(files,
                                msg="generate train_dataset",
                                n_mels=128,
                                n_frames=64,
                                n_hop_frames=8,
                                n_fft=1024,
                                hop_length=512,
                                power=2.0)
 
        # number of all files
        n_all_files = n_files_ea_section[section_idx]
        # number of vectors for each wave file
        n_vectors_ea_file = int(data.shape[0] / n_all_files)
        print(n_vectors_ea_file)
        
        # make one-hot vector for conditioning
        condition = np.ones((data.shape[0],2), float)
        condition[:,1] = 1
        n_vectors = n_vectors_ea_file * n_files_ea_section[section_idx]
 
        # 1D vector to 2D image
        data = data.reshape(data.shape[0], 64, 128, 1)
        print(data.shape)
 
        joblib.dump([data,condition], f"{BASE_DIR}Dev Data/dev_data_pump/data_{section_name}.jl", compress=3)


[1/1] /content/drive/MyDrive/DSN AI/Dev Data/dev_data_pump/pump
============== DATASET_GENERATOR ==============


generate train_dataset: 100%|██████████| 1003/1003 [03:52<00:00,  4.31it/s]


32
(32096, 64, 128, 1)


generate train_dataset: 100%|██████████| 1003/1003 [04:15<00:00,  3.93it/s]


32
(32096, 64, 128, 1)


generate train_dataset: 100%|██████████| 1003/1003 [04:13<00:00,  3.96it/s]


32
(32096, 64, 128, 1)


<Figure size 504x360 with 0 Axes>

### Loading Data for Training

In [ ]:
section_idx = 2
section_name = f"section_0{section_idx}"
machine_type = "pump"
ae_architecture = "lstm"
data, _ = joblib.load(f"{BASE_DIR}Dev Data/dev_data_pump/data_{section_name}.jl")

In [ ]:
if ae_architecture == "dense":
    data = data.reshape(32096, -1)
elif ae_architecture == "lstm":
    data = data.reshape(data.shape[:-1])
    data_norm = tf.keras.utils.normalize(data)
data.shape

(32096, 64, 128)

### Tuning

**Model Builders**

These help with setting up the search space for the tuning proces.

In [ ]:
def lstm_encoder_model_builder(hp):
    # define model
    timesteps = 64
    n_features = 128
    units1=hp.Int('units_1', 128, 2048, step=64)
    units2=hp.Int('units_2', 64, 1024, step=64)
 
 
    model = keras.Sequential()
    model.add(LSTM(units1, activation='relu', input_shape=(timesteps,n_features), return_sequences=True))
    model.add(LSTM(units2, activation='relu', return_sequences=False))
    model.add(RepeatVector(timesteps))
    model.add(LSTM(units2, activation='relu', return_sequences=True))
    model.add(LSTM(units1, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(n_features)))
    model.compile(optimizer='adam', loss='mse')
    
    return model
def lstm_model_builder(hp):
 
    # define model
    timesteps = 64
    n_features = 128
    units1=hp.Int('units_1', 128, 512, step=32)
    units2=hp.Int('units_2', 128, 256, step=32)
    
    model = keras.Sequential()
    model.add(LSTM(units1, activation='relu', input_shape=(timesteps,n_features), return_sequences=True))
    #model.add(LSTM(units2, activation='relu', return_sequences=True))
    model.add(LSTM(units2, activation='relu', return_sequences=True))
    model.add(LSTM(units1, activation='relu', return_sequences=True))
    model.add(Dense( n_features))
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
 
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mean_squared_error',
                metrics=['mse'])
    return model    
def dense_model_builder(hp):
    input_dim = 8192
    units1=hp.Int('units_1', 128, 2048, step=64)
    units2=hp.Int('units_2', 128, 1024, step=64)
    units3=hp.Int('units_3', 128, 512, step=64)
    units4=hp.Int('units_4', 8, 32, step=8)        # create model
    x = Input(shape=(input_dim,))
    h = Dense(units1)(x)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)
 
    h = Dense(units2)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)
 
    h = Dense(units3)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)
 
    h = Dense(units4)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)
    
    h = Dense(units3)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)
 
    h = Dense(units2)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)
 
    h = Dense(units1)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)
 
    h = Dense(input_dim)(h)
 
    model = Model(inputs=x, outputs=h)
 
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
 
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate,  ),
                loss='mean_squared_error',
                metrics=['mse'])
 
    return model

**Instantiate Bayesian Optimization Tuner**

In [ ]:
tuner = kt.tuners.bayesian.BayesianOptimization(lstm_model_builder,
                                                objective='val_mse',
                                                max_trials=5,
                                                num_initial_points=2,
                                                seed=s,
                                                directory='.',
                                                project_name=f'bayesian_{ae_architecture}',
                                                overwrite=True)

In [ ]:
 tuner.search(x=data_norm,
              y=data_norm,
            epochs= 1,
            shuffle=True,
            batch_size=512,
            validation_split=0.1           
            )
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
 
print(f"""
The hyperparameter search is complete.""")
best_hps.get_config()['values']

In [ ]:
 model = tuner.get_best_models()[0]

In [ ]:
### Load Model

model = keras.models.load_model('/content/drive/MyDrive/DSN_AI/Dev Data/dev_data_pump/pump/models/best_model.h5')

In [ ]:
### Continue Training

stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
checkpoint = keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/DSN_AI/Dev Data/dev_data_pump/pump/models/best_model.h5",
                                            verbose = 0, monitor = "val_mse",
                                            save_best_only=True,
                                            mode="auto", save_freq = 'epoch')
# Train the RNN
history = model.fit(x=data,
              y=data,
            epochs= 100,
            shuffle=True,
            validation_split=0.1,
            callbacks=[stop_early,checkpoint]
            )

### Fitting anomaly score distribution

In [ ]:
# calculate y_pred for fitting anomaly score distribution
y_pred = []
p = model.predict(data_norm)
ps = p.reshape(1003,32,64,128)
mfccs = data_norm.reshape(1003,32,64,128)
 
for mfcc,p in zip(mfccs,ps):
    y_pred.append(np.mean(np.square(mfcc - model.predict(p))))
 
print(np.mean(y_pred))
model_dir = "Models/MLP_Autoencoder/pump"
model_file_path = f"{BASE_DIR}{model_dir}/model_{machine_type}_{section_name}.hdf5"
 
# pickle file for storing section names
 
score_distr_file_path = f"{BASE_DIR}{model_dir}/score_distr_{machine_type}_{section_name}.pkl"
# fit anomaly score distribution
shape_hat, loc_hat, scale_hat = scipy.stats.gamma.fit(y_pred)
gamma_params = [shape_hat, loc_hat, scale_hat]
print(gamma_params)
joblib.dump(gamma_params, score_distr_file_path)
 
model.save(model_file_path)
print(f"save_model -> {model_file_path}")
print("============== END TRAINING ==============")

### Testing

In [ ]:
import scipy.stats
from sklearn import metrics
import csv
#machine type
machine_type = "pump"
 
# make output result directory
result_directory = f"{BASE_DIR}results/MLP_Autoencoder/{machine_type}"
# load base directory
model_dir = "Models/MLP_Autoencoder/pump"
dirs = [f"/content/drive/MyDrive/DSN_AI/Dev Data/dev_data_pump/{machine_type}"]
 
section_name = f"section_0{section_idx}"
# initialize lines in csv for AUC and pAUC
n_mels = 128
n_frames = 64
n_hop_frames = 8
n_fft = 10
hop_length = 512
power = 2.0
 
def save_csv(save_file_path,
             save_data):
    with open(save_file_path, "w", newline="") as f:
        writer = csv.writer(f, lineterminator='\n')
        writer.writerows(save_data)
 
performance_over_all = []
csv_lines = []
# loop of the base directory
for idx, target_dir in enumerate(dirs):
    print("\n===========================")
    print(f"[{idx+1}/{len(dirs)}] {target_dir}")
    machine_type = os.path.split(target_dir)[1]
 
    print("============== MODEL LOAD ==============")
    # load model file
    model_file = f"{BASE_DIR}{model_dir}/model_{machine_type}_{section_name}.hdf5"
    if not os.path.exists(model_file):
        print(f"{machine_type} model not found ")
 
    model = load_model(model_file)
    model.summary()
 
    # load section names for conditioning
    section_names_file_path = f"{BASE_DIR}{model_dir}/section_names_{machine_type}.pkl"
    n_sections = 3
    
    # load anomaly score distribution for determining threshold
    score_distr_file_path = f"{BASE_DIR}{model_dir}/score_distr_{machine_type}_{section_name}.pkl"
    shape_hat, loc_hat, scale_hat = joblib.load(score_distr_file_path)
 
    # determine threshold for decision
    decision_threshold = {0:0.999998768, 1: 0.57764,2:0.567035}
    decision_threshold = scipy.stats.gamma.ppf(q=decision_threshold[section_idx], a=shape_hat, loc=loc_hat, scale=scale_hat)
    print(decision_threshold)
 
    # results for each machine type
    csv_lines.append([machine_type])
    csv_lines.append(["section", "domain", "AUC", "pAUC", "precision", "recall", "F1 score"])
    performance = []
    dir_names = ["source_test", "target_test"]
    
    for dir_name in dir_names:
 
        #list machine id
        section_names = get_section_names(target_dir, dir_name=dir_name)
 
        section_name = section_names[section_idx]
 
        # load test file
        files, y_true = file_list_generator(target_dir=target_dir,
                                                section_name=section_name,
                                                dir_name=dir_name,
                                                mode=True)
 
        # setup anomaly score file path
        anomaly_score_csv = f"{result_directory}/anomaly_score_{machine_type}_{section_name}_{dir_name}.csv"
        anomaly_score_list = []
 
        # setup decision result file path
        decision_result_csv = f"{result_directory}/decision_result_{machine_type}_{section_name}_{dir_name}.csv"
 
        decision_result_list = []
 
        print("\n============== BEGIN TEST FOR A SECTION ==============")
        y_pred = [0. for k in files]
        pred = []
        for file_idx, file_path in tqdm(enumerate(files), total=len(files)):
            try:
                data = file_to_vectors(file_path,
                                           n_mels=n_mels,
                                           n_frames=n_frames,
                                           n_fft=n_fft,
                                           hop_length=hop_length,
                                           power=power)
            except:
                print(f"File broken!!: {file_path}")
 
            data = tf.keras.utils.normalize(data).reshape(-1,64, 128)
            p = model.predict(data)
            y_pred[file_idx] = np.mean(np.square(data - p))
            pred.append(np.mean(np.square(data - p)))
 
 
            # store anomaly scores
            anomaly_score_list.append([os.path.basename(file_path), y_pred[file_idx]])
 
            
 
            # store decision results
            if y_pred[file_idx] > decision_threshold:
                y_pred[file_idx] = 1
                decision_result_list.append([os.path.basename(file_path), 1])
            else:
                y_pred[file_idx] = 0
                decision_result_list.append([os.path.basename(file_path), 0])
        
        # output anomaly scores
        save_csv(save_file_path=anomaly_score_csv, save_data=anomaly_score_list)
        print(f"anomaly score result ->  {anomaly_score_csv}")
 
        # output decision results
        save_csv(save_file_path=decision_result_csv, save_data=decision_result_list)
        print(f"decision result ->  {decision_result_csv}")
 
        # append AUC and pAUC to lists
        
        print(metrics.accuracy_score(y_true, y_pred))
        auc = metrics.roc_auc_score(y_true, y_pred)
        p_auc = metrics.roc_auc_score(y_true, y_pred, max_fpr=0.1)
        tn, fp, fn, tp = metrics.confusion_matrix(y_true, [1 if x > decision_threshold else 0 for x in y_pred]).ravel()
        prec = tp / np.maximum(tp + fp, sys.float_info.epsilon)
        recall = tp / np.maximum(tp + fn, sys.float_info.epsilon)
        f1 = 2.0 * prec * recall / np.maximum(prec + recall, sys.float_info.epsilon)
        csv_lines.append([section_name.split("_", 1)[1], dir_name.split("_", 1)[0], auc, p_auc, prec, recall, f1])
        performance.append([auc, p_auc, prec, recall, f1])
        performance_over_all.append([auc, p_auc, prec, recall, f1])
        print(metrics.accuracy_score(y_true, y_pred))
        print(f"AUC : {auc}")
        print(f"pAUC : {p_auc}")
        print(f"precision : {prec}")
        print(f"recall : {recall}")
        print(f"F1 score : {f1}")
 
        print("\n============ END OF TEST FOR A SECTION ============")
 
 
# calculate averages for AUCs and pAUCs
amean_performance = np.mean(np.array(performance, dtype=float), axis=0)
csv_lines.append(["arithmetic mean", ""] + list(amean_performance))
hmean_performance = scipy.stats.hmean(np.maximum(np.array(performance, dtype=float), sys.float_info.epsilon), axis=0)
csv_lines.append(["harmonic mean", ""] + list(hmean_performance))
csv_lines.append([])
 
clear_session()
gc.collect()
 
csv_lines.append(["", "", "AUC", "pAUC", "precision", "recall", "F1 score"])
# calculate averages for AUCs and pAUCs
amean_performance = np.mean(np.array(performance_over_all, dtype=float), axis=0)
csv_lines.append(["arithmetic mean over all machine types, sections, and domains", ""] + list(amean_performance))
hmean_performance = scipy.stats.hmean(np.maximum(np.array(performance_over_all, dtype=float), sys.float_info.epsilon), axis=0)
csv_lines.append(["harmonic mean over all machine types, sections, and domains", ""] + list(hmean_performance))
csv_lines.append([])
 
# output results
result_path = f"{result_directory}/result.csv"
print("results -> {}".format(result_path))
save_csv(save_file_path=result_path, save_data=csv_lines)

Tuning the Threshold

In [ ]:
store = {"thresh":0, "acc":0, "dt":0}
dt = 0.56
while dt< 0.57:
  decision_threshold = scipy.stats.gamma.ppf(q=dt, a=shape_hat, loc=loc_hat, scale=scale_hat)
  y_pred = (np.array(pred)>decision_threshold).astype(int)
  if metrics.accuracy_score(y_true,y_pred)> store["acc"]:
    store["acc"] =metrics.accuracy_score(y_true,y_pred)
    store["thresh"] = decision_threshold
    store["dt"]  = dt
  dt+=0.000001
 
print(store)

{'thresh': 0.0037117979606261244, 'acc': 0.535, 'dt': 0.5670350000002023}


### Single Track Inference

In [ ]:
#%%writefile /content/drive/MyDrive/DSN_AI/Models/MLP_Autoencoder/inference.py
#import libraries
import librosa
import requests
import os
import sys
import numpy as np
import joblib
import tensorflow as tf
import scipy.stats
 
from cloud_client import *
 
section_idx = 0
machine_type = "pump"
pump_container = f"{machine_type}-section-0{section_idx}"
file_path = download_file(pump_container)
section_name = f"section_0{section_idx}"
 
 
# constants
SAMPLE_RATE = 22050
 
model_dir = f"{BASE_DIR}Models/MLP_Autoencoder/{machine_type}"
 
# load anomaly score distribution for determining threshold
score_distr_file_path = f"{model_dir}/score_distr_{machine_type}_{section_name}.pkl"
shape_hat, loc_hat, scale_hat = joblib.load(score_distr_file_path)
 
decision_threshold = {0: 0.999998768, 1:0.57764, 2:0.567035}
decision_threshold = scipy.stats.gamma.ppf(q=decision_threshold[section_idx], a=shape_hat, loc=loc_hat, scale=scale_hat)
 
 
# feature extractor
def file_to_vectors(file_path,
                    n_mels=64,
                    n_frames=5,
                    n_fft=1024,
                    hop_length=512,
                    power=2.0):
    """
    convert file_name to a vector array.
    file_name : str
        target .wav file
    return : numpy.array( numpy.array( float ) )
        vector array
        * dataset.shape = (dataset_size, feature_vector_length)
    """
    # calculate the number of dimensions
    dims = n_mels * n_frames
 
    # generate melspectrogram using librosa
    signal, sr = librosa.load(file_path, sr=None, mono=True)
    mel_spectrogram = librosa.feature.melspectrogram(y=signal,
                                                     sr=sr,
                                                     n_fft=n_fft,
                                                     hop_length=hop_length,
                                                     n_mels=n_mels,
                                                     power=power)
 
    # convert melspectrogram to log mel energies
    log_mel_spectrogram = 20.0 / power * np.log10(np.maximum(mel_spectrogram, sys.float_info.epsilon))
    # calculate total vector size
    n_vectors = len(log_mel_spectrogram[0, :]) - n_frames + 1
    # skip too short clips
    if n_vectors < 1:
        return np.empty((0, dims))
 
    # generate feature vectors by concatenating multiframes
    vectors = np.zeros((n_vectors, dims))
    for t in range(n_frames):
        vectors[:, n_mels * t : n_mels * (t + 1)] = log_mel_spectrogram[:, t : t + n_vectors].T
    # delete audio file
    os.remove(file_path)
    return vectors
 
# load model file
model_file = f"{model_dir}/model_{machine_type}_{section_name}.hdf5"
if not os.path.exists(model_file):
    print(f"{machine_type} model not found ")
 
model = tf.keras.models.load_model(model_file)
 
# make prediction
data = file_to_vectors(file_path, 128, 64).reshape(-1,64,128)
 
data_norm = tf.keras.utils.normalize(data)
p = model.predict(data_norm)
               
y_pred = np.mean(np.square(data_norm - p))
if y_pred > decision_threshold:
    decision_result = [os.path.basename(file_path), 1]
else:
    decision_result = [os.path.basename(file_path), 0]
print(decision_result)